# Cleaning US Census Data

You just got hired as a Data Analyst at the Census Bureau, which collects census data and creates interesting visualizations and insights from it.

The person who had your job before you left you all the data they had for the most recent census. It is in multiple `csv` files. They didn't use pandas, they would just look through these `csv` files manually whenever they wanted to find something. Sometimes they would copy and paste certain numbers into Excel to make charts.

The thought of it makes you shiver. This is not scalable or repeatable.

Your boss wants you to make some scatterplots and histograms by the end of the day. Can you get this data into `pandas` and into reasonable shape so that you can make these histograms?

## Inspect the Data!

1. The first visualization your boss wants you to make is a scatterplot that shows average income in a state vs proportion of women in that state.

   Open some of the census `csv` files that came with the kit you downloaded. How are they named? What kind of information do they hold? Will they help us make this graph?

Columns: State, TotalPop, Hispanic, White, Black, Native, Asian, Pacific, Income, GenderPop
         string, number, string (because of "%") X 6, string (because of "$"), string with format xxxM_xxxF

2. It will be easier to inspect this data once we have it in a DataFrame. You can't even call `.head()` on these `csv`s! How are you supposed to read them?

   Using `glob`, loop through the census files available and load them into DataFrames. Then, concatenate all of those DataFrames together into one DataFrame, called something like `us_census`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

# Reading the data
files = glob.glob("states*.csv")

df_list = []
for filename in files:
  data = pd.read_csv(filename)
  df_list.append(data)

us_census = pd.concat(df_list)


3. Look at the `.columns` and the `.dtypes` of the `us_census` DataFrame. Are those datatypes going to hinder you as you try to make histograms?

In [ ]:
# Checking what the data looks like
print(us_census.columns)
print(us_census.dtypes)


Most of the data needs to be modified by removing "%" and "$" and making them into floats

4. Look at the `head()` of the DataFrame so that you can understand why some of these `dtypes` are objects instead of integers or floats.

   Start to make a plan for how to convert these columns into the right types for manipulation.

In [ ]:
print(us_census.head())

## Regex to the Rescue

5. Use regex to turn the `Income` column into a format that is ready for conversion into a numerical type.

In [ ]:
# Removing "$" from the "Income" column
import re
us_census.Income = us_census.Income.str.replace("$", "", regex=False)
# Making it numerical (changing the formatting due to local formatting being different)
us_census.Income = us_census.Income.str.replace(",", "", regex=False)
us_census.Income = pd.to_numeric(us_census.Income)

6. Look at the `GenderPop` column. We are going to want to separate this into two columns, the `Men` column, and the `Women` column.

   Split the column into those two new columns using `str.split` and separating out those results.

In [ ]:
# Splitting the "GenderPop" into "men" and "Women" columns
split_string = us_census.GenderPop.str.split("_", expand=True)
us_census["Men"] = split_string[0]
us_census["Women"] = split_string[1]
print(us_census.head())

7. Convert both of the columns into numerical datatypes.

   There is still an `M` or an `F` character in each entry! We should remove those before we convert.

In [ ]:
# Removing "M" and "F" from the gender columns
us_census.Men = us_census.Men.str.replace("M", "", regex=False)
us_census.Women = us_census.Women.str.replace("F", "", regex=False)

# Making them numerical
us_census.Men = pd.to_numeric(us_census.Men)
us_census.Women = pd.to_numeric(us_census.Women)
print(us_census.head())

8. Now you should have the columns you need to make the graph and make sure your boss does not slam a ruler angrily on your desk because you've wasted your whole day cleaning your data with no results to show!

   Use matplotlib to make a scatterplot!
   
   ```py
   plt.scatter(the_women_column, the_income_column)
   ```
   
   Remember to call `plt.show()` to see the graph!

In [ ]:
# Making a scatterplot
plt.scatter(us_census.Women, us_census.Income) 
plt.show()


9. You want to double check your work. You know from experience that these monstrous csv files probably have `nan` values in them! Print out your column with the number of women per state to see.

   We can fill in those `nan`s by using pandas' `.fillna()` function.
   
   You have the `TotalPop` per state, and you have the `Men` per state. As an estimate for the `nan` values in the `Women` column, you could use the `TotalPop` of that state minus the `Men` for that state.
   
   Print out the `Women` column after filling the `nan` values to see if it worked!

In [ ]:
# Checking for lacking data in "Women" column
print("This dataset has null values in 'Women' column: ", us_census.Women.isnull().any())

# Filling in the missing values
us_census.Women = us_census.Women.fillna(us_census.TotalPop-us_census.Men)


10. We forgot to check for duplicates! Use `.duplicated()` on your `census` DataFrame to see if we have duplicate rows in there.

In [ ]:
# Checking for duplicates
print("There are duplicate rows in this data set: ", us_census.duplicated().any())


11. Drop those duplicates using the `.drop_duplicates()` function.

In [ ]:
# Dropping the non-existent duplicates
us_census = us_census.drop_duplicates()


12. Make the scatterplot again. Now, it should be perfect! Your job is secure, for now.

In [ ]:
# Making a scatterplot
plt.scatter(us_census.Women, us_census.Income) 
plt.show()


## Histogram of Races

13. Now your boss wants you to make a bunch of histograms out of the race data that you have. Look at the `.columns` again to see what the race categories are.

In [ ]:
# Checking the columns
print(us_census.columns)


14. Try to make a histogram for each one!

    You will have to get the columns into the numerical format, and those percentage signs will have to go.
    
    Don't forget to fill the `nan` values with something that makes sense! You probably dropped the duplicate rows when making your last graph, but it couldn't hurt to check for duplicates again.

In [ ]:
# Removing "%" from the race columns and making them numerical
for col in ['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']:
    us_census[col] = us_census[col].str.replace('%', '', regex=False)
    us_census[col] = pd.to_numeric(us_census[col])

# Checking for null values
print("This dataset has null values: ", us_census.isnull().any().any())

# Checking where they are
print("Locations of null values:\n")
#print("Rows:", us_census.isnull().any(axis=1))
print("Columns:", us_census.isnull().any(axis=0))

# Adding the missing data in the "Pacific" column
us_census.Pacific = us_census.Pacific.fillna(100-us_census.Hispanic-us_census.White-us_census.Black-us_census.Native-us_census.Asian)

# Plotting the race bar chart
us_census[['Hispanic', 'White', 'Black', "Native", "Asian", "Pacific"]].plot(
    kind='bar', stacked=True, figsize=(10,6))
plt.ylabel('Percentage')
plt.title('Ethnic Backgrounds by Area')
plt.show()

## Get Creative

15. Phew. You've definitely impressed your boss on your first day of work.

    But is there a way you really convey the power of pandas and Python over the drudgery of `csv` and Excel?
    
    Try to make some more interesting graphs to show your boss, and the world! You may need to clean the data even more to do it, or the cleaning you have already done may give you the ease of manipulation you've been searching for.

In [ ]:
# Plotting distributions of the ethnic backgrounds, one per dubplot
us_census[['Hispanic', 'White', 'Black', "Native", "Asian", "Pacific"]].hist(bins=20, figsize=(12,8))
plt.suptitle('Distribution of Ethnic Background Percentages')
plt.show()
